### Melon 100 Chart 
* 100곡의 노래의 제목, Song ID추출 (정규표현식 사용)
* Song ID로 상세페이지를 100번 요청해서 노래 상세정보 추출
* 100곡의 상세정보를 json file에 저장한다.
* json file을 Pandas의 DataFrame 객체에 저장해서 표 데이터를 만들기
* 표 데이터를 MariaDB에 저장하기

#### 1.100곡의 노래의 제목, Song ID추출 (정규표현식 사용)

In [20]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 요청을 보내기
res = requests.get(url, headers=req_header)
print(res.status_code)
# 응답결과가 정상이면
if res.ok:
    # 응답 객체에 포함된 text를 추출
    html = res.text
    # 추출한 text를 파싱하기 위한 BeautifulSoup 객체생성
    soup = BeautifulSoup(html, 'html.parser')
    # <div id='tb_list'> div 태그 선택 ,1개
    print(len(soup.select('div#tb_list')))
    # <div id='tb_list'> 하위에 있는 tr 태그 선택 ,101개
    print(len(soup.select('div#tb_list tr')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> , 200개
    print(len(soup.select('div#tb_list tr .wrap_song_info')))
    print(len(soup.select('div.wrap_song_info')))
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> 하위에 a
    print(len(soup.select('div#tb_list tr .wrap_song_info a'))) #458
    print(len(soup.select('div.wrap_song_info a'))) #458
    # <div id='tb_list'> 하위에 있는 tr 하위에 <div class='wrap_song_info'> 
    # 하위에 <a href='playSong'> 
    print(len(soup.select('div#tb_list tr .wrap_song_info a[href*=\'playSong\']')))
    
    atag_list = soup.select("div#tb_list tr .wrap_song_info a[href*='playSong']")
    
    # 100곡 노래 저장할 list
    song_list = []
    for idx, atag in enumerate(atag_list, 1):
        # 1곡 노래 저장할 dict
        song_dict = {}
        
        # a tag 의 content
        title = atag.text
        # a tag의 href 속성 값
        href = atag['href']
        # 정규표현식 패턴으로 숫자를 검색
        matched = re.search(r'(\d+)\);', href)
        #print(matched, matched.group(0), matched.group(1))
        song_id = matched.group(1)
        print(idx, title, song_id)
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        

200
1
101
200
200
458
458
100
1 VVS (Feat. JUSTHIS) (Prod. GroovyRoom) 33077590
2 밤하늘의 별을(2020) 33061995
3 Dynamite 32872978
4 잠이 오질 않네요 33013877
5 내일이 오면 (Feat. 기리보이, BIG Naughty (서동현)) 33115807
6 힘든 건 사랑이 아니다 32998018
7 Lovesick Girls 32961718
8 취기를 빌려 (취향저격 그녀 X 산들) 32794652
9 Life Goes On 33077234
10 오래된 노래 3894276
11 CREDIT (Feat. 염따, 기리보이, Zion.T) 33150997
12 Savage Love (Laxed - Siren Beat) (BTS Remix) 32962258
13 내 마음이 움찔했던 순간 (취향저격 그녀 X 규현) 32871975
14 나랑 같이 걸을래 (바른연애 길잡이 X 적재) 33011180
15 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 32061975
16 Achoo (Feat. pH-1, HAON) (Prod. GroovyRoom) 33115804
17 에잇(Prod.&Feat. SUGA of BTS) 32578498
18 ON AIR (Feat. 로꼬, 박재범 & GRAY) 33150993
19 Freak (Prod. Slom) 33077591
20 When We Disco (Duet with 선미) 32853712
21 혼술하고 싶은 밤 33104090
22 흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야 32003395
23 늦은 밤 너의 집 앞 골목길에서 32156286
24 어떻게 지내 (Prod. By VAN.C) 32438894
25 뿌리 (Feat. JUSTHIS) (Prod. GroovyRoom) 33115806
26 딩가딩가 (Dingga) 33001672
27 How You Like That 32720013
28 모든 날, 모든 순간 (Ever